In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/enron-email-dataset/emails.csv


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import email
import re

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = stopwords.words('english') 

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
data = pd.read_csv("/kaggle/input/enron-email-dataset/emails.csv")

data.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [4]:
catagories = []
for name in data['file']:
  name=name.split('/')
  catagories.append(name[1])

data['catagory'] = catagories
labels = list(data['catagory'].unique())

print(data['catagory'].value_counts())

all_documents                       128103
discussion_threads                   58609
sent                                 57653
deleted_items                        51356
inbox                                44859
                                     ...  
synfuel                                  1
associations                             1
black_hills                              1
government_affairs_group_reports         1
sithe                                    1
Name: catagory, Length: 1427, dtype: int64


In [5]:
selected_catagories =  ['discussion_threads', 'personal', 'meetings', 'logistics', 'calendar', 'archiving', 'california', 'power', 'deal_communication', 'resumes']

data = data[data['catagory'].isin(selected_catagories)]

In [6]:
#mail body
email_body = []

for mail in data['message']:
  mail = email.message_from_string(mail)  

  # getting message body  
  message_body = mail.get_payload()
  
  email_body.append(message_body)

data['message_body'] = email_body


#getting headers

headers = {"Date":[], "Subject":[], "X-Folder":[], "X-From":[], "X-To":[]}
for mail in data['message']:
  mail = email.message_from_string(mail)  

  #get other headers
  for header in headers.keys():
    headers[header].append(mail.get(header))

for key in headers.keys():
  data[key] = headers[key]
data['Date'] = pd.to_datetime(data['Date'])

print(data.head())

                                 file  \
1593    allen-p/discussion_threads/1.   
1594   allen-p/discussion_threads/10.   
1595  allen-p/discussion_threads/100.   
1596  allen-p/discussion_threads/101.   
1597  allen-p/discussion_threads/102.   

                                                message            catagory  \
1593  Message-ID: <20379972.1075855673249.JavaMail.e...  discussion_threads   
1594  Message-ID: <21833620.1075855673443.JavaMail.e...  discussion_threads   
1595  Message-ID: <25754034.1075855675456.JavaMail.e...  discussion_threads   
1596  Message-ID: <29444139.1075855675477.JavaMail.e...  discussion_threads   
1597  Message-ID: <6888095.1075855675499.JavaMail.ev...  discussion_threads   

                                           message_body  \
1593  Naomi,\n\nThe two analysts that I have had con...   
1594  Jeff/Brenda:\n\nPlease authorize the following...   
1595  ---------------------- Forwarded by Phillip K ...   
1596  ---------------------- Forwarded by 

In [7]:
print(data.shape)

(78251, 9)


In [8]:
data.dropna(inplace = True)
print(data.shape)

(78232, 9)


In [9]:
#Removing non alphanumeric characters
def clean_text(text):
  cleanText = text.lower()
  cleanText = re.sub(r'[\W\d]', " ", cleanText)  

  return cleanText

#Removing stop words
def stop_word_removal(text):
  tokens = [token for token in text if token not in stopwords]
  return tokens

#Tokenizing
def tokenize(text):
  tokens = text.split(" ")
  return tokens

headers = ["Subject", "X-Folder", "X-From", "X-To", "message_body"]

tokens = [] 

for i in range(data.shape[0]):
  tokens_i = []
  for header in headers:
    tokens_h = clean_text(data[header].values[i])
    tokens_h = tokenize(tokens_h)
    tokens_h = stop_word_removal(tokens_h)

    tokens_i.extend(tokens_h)

  tokens.append(tokens_i)

In [10]:
for i in range(len(tokens)):
  tokens[i].remove("")
  tokens[i] = " ".join(tokens[i])

data['final_text'] = tokens

print(data['final_text'])

1593       phillip_allen_dec     notes folders discussio...
1594      eol phillip_allen_dec     notes folders discus...
1595      ena management committee phillip_allen_dec    ...
1596      daily report phillip_allen_dec     notes folde...
1597      daily report phillip_allen_dec     notes folde...
                                ...                        
516850    updated edcc ecc pricing discussion  exmerge  ...
516851    aes project  tolling interest  exmerge   zuffe...
516852    transfer enron direct contracts ed  marking in...
516971    momentum motor cars john_zufferlie_dec     not...
516972    new email address john_zufferlie_dec     notes...
Name: final_text, Length: 78232, dtype: object


In [18]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(data['final_text'], data['catagory'], test_size=0.3)

In [12]:
# x_train, x_val, y_train, y_val = model_selection.train_test_split(x_train, y_train, test_size=0.4)

In [19]:
Encoder = LabelEncoder()
y_train = Encoder.fit_transform(y_train)
y_test = Encoder.fit_transform(y_test)

In [14]:
# y_val = Encoder.fit_transform(y_val)

In [20]:
Tfidf_vect = TfidfVectorizer()

Tfidf_vect.fit(data['final_text'])

x_train = Tfidf_vect.transform(x_train)
x_test = Tfidf_vect.transform(x_test)
# x_val = Tfidf_vect.transform(x_val)

In [21]:
Naive = naive_bayes.MultinomialNB()
Naive.fit(x_train, y_train)

predictions_NB = Naive.predict(x_test)
print("Naive Bayes Accuracy Score -> ", accuracy_score(predictions_NB, y_test)*100)

Naive Bayes Accuracy Score ->  78.18065615679592


In [22]:
SVM = svm.SVC(C=1, kernel='linear', degree=3, gamma='auto')
SVM.fit(x_train, y_train)

predictions_SVM = SVM.predict(x_test)
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, y_test)*100)

SVM Accuracy Score ->  94.65274818917767
